Adaptado de [Machine Learning in Production](https://www.deeplearning.ai/courses/machine-learning-in-production/) de [Andrew Ng](https://www.deeplearning.ai/)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/))

# Servir um modelo com o TensorFlow Serving
------------------------

Neste laboratório, você dará uma olhada no sistema de fornecimento de modelos da TFX para produção chamado [Tensorflow Serving](https://www.tensorflow.org/tfx/guide/serving). Esse sistema é altamente integrado ao Tensorflow e oferece uma maneira fácil e direta de implantar modelos.

Especificamente, você irá:
- Aprender a instalar o TF Serving.
- Carregar um modelo pré-treinado que classifica cães, pássaros e gatos.
- Salve-o seguindo as convenções necessárias para o TF serving.
- Rodar um servidor Web usando o TF serving que aceitará solicitações por HTTP.
- Ingeragir com seu modelo por meio de uma API REST.
- Saiba mais sobre o controle de versão do modelo.

Este laboratório se inspira [nesse tutorial oficial do Tensorflow](https://www.tensorflow.org/tfx/tutorials/serving/rest_simple), portanto, consulte-o se tiver dúvidas sobre os tópicos abordados aqui.

Observe que, diferentemente do último laboratório, você trabalhará com o TF servindo sem usar o Docker. O objetivo é mostrar a você as diferentes maneiras de usar esse sistema de serviço.

Vamos começar!

### Importações

In [ ]:
import os
import zipfile
import subprocess
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import Image, display

### Baixando os dados

Neste laboratório, você não treinará um modelo, mas usará um modelo para obter previsões, por isso precisa de algumas imagens de teste. O modelo que você usará foi originalmente treinado com imagens dos conjuntos de dados *cats and dogs* e  *Caltech birds*. Para solicitar previsões, são fornecidas algumas imagens do conjunto de teste:

In [ ]:
# Baixar as imagens
!wget -q https://storage.googleapis.com/mlep-public/course_3/week2/images.zip

# Definir o diretório base
base_dir = '/tmp/data'

# Descompactar as imagens
with zipfile.ZipFile('/content/images.zip', 'r') as my_zip:
  my_zip.extractall(base_dir)

# Salvar os caminhos para as imagens de cada classe
dogs_dir = os.path.join(base_dir, 'images/dogs')
cats_dir = os.path.join(base_dir,'images/cats')
birds_dir = os.path.join(base_dir,'images/birds')

# Imprimir a quantidade de imagens de cada classe
print(f"Há {len(os.listdir(dogs_dir))} imagens de cachorros")
print(f"Há {len(os.listdir(cats_dir))} imagens de gatos")
print(f"Há {len(os.listdir(birds_dir))} imagens de pássaros\n\n")

# Veja exemplos de imagens de cada classe
print("Exemplo de imagem de gato:")
display(Image(filename=f"{os.path.join(cats_dir, os.listdir(cats_dir)[0])}"))
print("\nExemplo de imagen de cachorro:")
display(Image(filename=f"{os.path.join(dogs_dir, os.listdir(dogs_dir)[0])}"))
print("\nExemplo de imagem de pássaro:")
display(Image(filename=f"{os.path.join(birds_dir, os.listdir(birds_dir)[0])}"))

Agora que você está familiarizado com os dados com os quais trabalhará, vamos passar para o modelo.

### Carregar um modelo pré-treinado

O objetivo deste laboratório é demonstrar os recursos do TF serving, portanto, você não investirá tempo treinando um modelo. Em vez disso, você usará um modelo pré-treinado. Esse modelo classifica imagens de pássaros, gatos e cachorros e foi treinado com aumento de imagem para produzir resultados muito bons.

Primeiro, faça o download dos arquivos necessários:

In [ ]:
!wget -q -P /content/model/ https://storage.googleapis.com/mlep-public/course_1/week2/model-augmented/saved_model.pb
!wget -q -P /content/model/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-augmented/variables/variables.data-00000-of-00001
!wget -q -P /content/model/variables/ https://storage.googleapis.com/mlep-public/course_1/week2/model-augmented/variables/variables.index

Agora, carregue o modelo na memória:

In [ ]:
model = tf.keras.models.load_model('/content/model')

Neste ponto, você pode presumir que treinou o modelo com sucesso. Você pode ignorar os avisos sobre o modelo ter sido treinado em uma versão mais antiga do TensorFlow.

Para fins de contexto, esse modelo usa uma arquitetura CNN simples. Dê uma olhada rápida nas camadas que o compõem:

In [ ]:
model.summary()

## Salve seu modelo

Para carregar nosso modelo treinado no TensorFlow Serving, primeiro precisamos salvá-lo no formato [SavedModel](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/saved_model).

Isso criará um arquivo [protobuf](https://developers.google.com/protocol-buffers) em uma hierarquia de diretórios bem definida e incluirá um número de versão.

O [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving) nos permite selecionar qual versão de um modelo, ou "servível", queremos usar quando fizermos solicitações de inferência.

Cada versão será exportada para um subdiretório diferente no caminho fornecido.

In [ ]:
# Buscar a sessão do Keras e salvar o modelo
# A definição da assinatura é definida pelos tensores de entrada e saída,
# e armazenada com a chave de serviço padrão
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print(f'export_path = {export_path}\n')


# Salvar o modelo
tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

Um modelo salvo em disco inclui os seguintes arquivos:
- `assets`: um diretório que inclui arquivos arbitrários usados pelo grafo TF.
- `variables`: um diretório que contém informações sobre os pontos de verificação de treinamento do modelo.
- `saved_model.pb`: o arquivo protobuf que representa o programa TF real.

Dê uma olhada rápida nesses arquivos:

In [ ]:
print(f'\nArquivos do modelo salvos em {export_path }:\n')
!ls -lh {export_path}

## Examine seu modelo salvo

Usaremos o utilitário de linha de comando `saved_model_cli` para examinar os [MetaGraphDefs](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/MetaGraphDef) (os modelos) e [SignatureDefs](https://www.tensorflow.org/tfx/serving/signature_defs) (os métodos que você pode chamar) em nosso SavedModel.

Para maiores detalhes consulte [essa discussão sobre a CLI do SavedModel](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/saved_model.md#cli-to-inspect-and-execute-savedmodel) no Guia do TensorFlow.

In [ ]:
!saved_model_cli show --dir {export_path} --tag_set serve --signature_def serving_default

Isso nos diz muito sobre o nosso modelo!

Nesse caso, não treinamos explicitamente o modelo, portanto, qualquer informação sobre as entradas e saídas é muito valiosa. 

Por exemplo, sabemos que esse modelo espera que nossas entradas tenham a forma `(150, 150, 3)`, o que, combinado com o uso de camadas `conv2d`, sugere que esse modelo espera imagens coloridas em uma resolução de `150 por 150`.

Além disso, a saída do modelo tem a forma `(3)`, o que sugere uma ativação `softmax` com 3 classes.

## Preparar dados para inferência

Agora que você conhece a forma dos dados esperados pelo modelo, é hora de pré-processar as imagens de teste adequadamente. 

Essas imagens são fornecidas em uma ampla variedade de resoluções, e felizmente o Keras tem o que você precisa com seu [`ImageDataGenerator`](https://keras.io/api/preprocessing/image/).

Usando esse objeto, você pode:
- Normalizar valores de pixel.
- Padronizar resoluções de imagem.
- Definir um tamanho de lote para inferência.
- E muito mais!

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalizar os valores dos pixels
test_datagen = ImageDataGenerator(rescale=1./255)

# Apontar para o diretório com as imagens de teste
val_gen_no_shuffle = test_datagen.flow_from_directory(
    '/tmp/data/images',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=True)

# Imprimir o rótulo associado a cada classe
print(f"Os rótulos de cada classe no gerador de testes são: {val_gen_no_shuffle.class_indices}")

Como esse objeto é um [gerador](https://wiki.python.org/moin/Generators), você pode obter um lote de imagens e rótulos usando a função `next`:

In [ ]:
# Obtenha um lote de 32 imagens junto com seu rótulo verdadeiro
data_imgs, labels = next(val_gen_no_shuffle)

# Verifique os formatos
print(f"data_imgs tem formato: {data_imgs.shape}")
print(f"os rótulos tem o formato: {labels.shape}")

Como esperado, `data_imgs` é uma matriz que contém 32 imagens coloridas com resolução de 150x150. De maneira semelhante, `labels` tem o rótulo verdadeiro para cada uma dessas 32 imagens.

Para verificar se tudo está funcionando corretamente, faça uma verificação de sanidade para plotar as primeiras 5 imagens do lote:

In [ ]:
from tensorflow.keras.preprocessing.image import array_to_img

# Retorna a representação em string de cada classe
def get_class(index):
  if index == 0:
    return "bird"
  elif index == 1:
    return "cat"
  elif index == 2:
    return "dog"
  return None


# Plota uma matriz numpy que representa uma imagem
def plot_array(array, label, pred=None):
  array = np.squeeze(array)
  img = array_to_img(array)
  display(img)
  if pred is None:
    print(f"A imagem mostra um {get_class(label)}.\n")
  else:
    print(f"A imagem mostra um {get_class(label)}. O model prediz {get_class(pred)}.\n")


# Plotar 5 imagens do lote
for i in range(5):
  plot_array(data_imgs[i], labels[i])

Todas as imagens têm a mesma resolução e os rótulos verdadeiros estão corretos.

Vamos começar a servir o modelo!

## Sirva seu modelo com o TensorFlow Serving

### Instalar o TensorFlow Serving

Você precisará instalar uma versão mais antiga (2.8.0) porque as versões mais recentes são atualmente incompatíveis com o Colab.

In [ ]:
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-universal-2.8.0/t/tensorflow-model-server-universal/tensorflow-model-server-universal_2.8.0_all.deb'
!dpkg -i tensorflow-model-server-universal_2.8.0_all.deb

### Iniciar a execução do TensorFlow Serving

É aqui que começamos a executar o TensorFlow Serving e carregamos nosso modelo.  Depois que ele for carregado, poderemos começar a fazer solicitações de inferência usando REST.  Há alguns parâmetros importantes:

* `rest_api_port`: A porta que você usará para solicitações REST.
* `model_name`: Você o usará no URL das solicitações REST.  Pode ser qualquer coisa. Nesse caso, é usado `animal_classifier`.
* `model_base_path`: É o caminho para o diretório em que você salvou o modelo.

In [ ]:
# Defina uma variável env com o caminho para onde o modelo é salvo
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
# Ativar o servidor de serviço TF
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=animal_classifier \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Dê uma olhada no final dos registros impressos do meu servidor modelo TF:

In [ ]:
!tail server.log

O servidor foi capaz de carregar e servir o modelo com sucesso!

Como você vai interagir com o servidor por meio de HTTP/REST, deve apontar as solicitações para `localhost:8501`, como está sendo impresso nos logs acima.

## Faça uma solicitação ao seu modelo no TensorFlow Serving

Neste ponto, você já sabe como são seus dados de teste. Você fará previsões para imagens coloridas de 150x150 em lotes de 32 imagens (representadas por matrizes numéricas) de cada vez.

Como o REST espera que os dados estejam no formato JSON e o JSON não é compatível com tipos de dados Python personalizados, como matrizes numpy, primeiro você precisa converter essas matrizes em listas aninhadas.

O serviço TF espera um campo chamado `instances` que contém os tensores de entrada para o modelo. Para passar seus dados para o modelo, você deve criar um JSON com seus dados como valor para a chave `instances`.

In [ ]:
import json

# Converter matriz numpy em lista
data_imgs_list = data_imgs.tolist()

# Create JSON to use in the request
data = json.dumps({"instances": data_imgs_list})

### Fazer solicitações REST

Enviaremos uma solicitação de previsão como uma solicitação POST para o ponto de extremidade REST do nosso servidor e passaremos a ele o lote de 32 imagens. 

Lembre-se de que o endpoint que serve o modelo está localizado em `http://localhost:8501`. No entanto, esse URL ainda precisa de alguns parâmetros adicionais para tratar adequadamente a solicitação. Você deve anexar `v1/models/name-of-your-model:predict` a ele para que o TF serving saiba qual modelo procurar e executar uma tarefa de previsão.

Você também deve passar para a solicitação os dados que contêm a lista que representa as 32 imagens, juntamente com um dicionário de cabeçalhos que especifica o tipo de conteúdo que será passado, que, nesse caso, é JSON.

Depois de obter uma resposta do servidor, você pode obter as previsões dela inspecionando o campo `predictions` do JSON que a resposta retornou.

In [ ]:
import requests

# Definir cabeçalhos com o tipo de conteúdo definido como json
headers = {"content-type": "application/json"}

# Capture a resposta fazendo uma solicitação para o URL apropriado com os parâmetros apropriados
json_response = requests.post('http://localhost:8501/v1/models/animal_classifier:predict', data=data, headers=headers)

# Analisar as previsões a partir da resposta
predictions = json.loads(json_response.text)['predictions']

# Imprimir a forma das previsões
print(f"predictions has shape: {np.asarray(predictions).shape}")

Você pode achar estranho que as previsões tenham retornado 3 valores para cada imagem. No entanto, lembre-se de que a última camada do modelo é uma função `softmax`, portanto, ela retornou um valor para cada uma das classes. Para obter as previsões reais, você precisa encontrar o argumento máximo:

In [ ]:
# Calcule argmax
preds = np.argmax(predictions, axis=1)

# Imprima o formato das predições
print(f"preds has shape: {preds.shape}")

Agora você tem uma classe prevista para cada uma das imagens de teste! Muito bom!

Para testar a qualidade do desempenho do modelo, vamos plotar as primeiras 10 imagens com os rótulos verdadeiros e previstos:

In [ ]:
for i in range(10):
  plot_array(data_imgs[i], labels[i], preds[i])

Para fazer mais testes, você pode plotar mais imagens das 32 ou até mesmo tentar gerar um novo lote a partir do gerador e repetir as etapas acima.

## Desafio opcional

Tente recriar as etapas acima para o próximo lote de 32 imagens:

In [ ]:
# Insira seu código aqui



## Solução

Se você quiser alguma ajuda, a resposta pode ser encontrada na próxima célula:

In [ ]:
# Obter um lote de 32 imagens junto com seu rótulo verdadeiro
data_imgs, labels = next(val_gen_no_shuffle)

# Converter matriz numérica em lista
data_imgs_list = data_imgs.tolist()

# Criar JSON para usar na solicitação
data = json.dumps({"instances": data_imgs_list})

# Capture a resposta fazendo uma solicitação para o URL apropriado com os parâmetros apropriados
json_response = requests.post('http://localhost:8501/v1/models/animal_classifier:predict', data=data, headers=headers)

# Analisar as previsões a partir da resposta
predictions = json.loads(json_response.text)['predictions']

# Calcule argmax
preds = np.argmax(predictions, axis=1)

for i in range(5):
  plot_array(data_imgs[i], labels[i], preds[i])

# Conclusão
**Parabéns por terminar este laboratório!**

Agora você deve ter uma compreensão mais profunda dos aspectos internos do TF serving. No laboratórioanterior, você viu como usar o TFS junto com o Docker. Neste laboratório, você viu como o TFS e o `tensorflow-model-server` funcionam sozinhos. Você também viu como salvar um modelo e a estrutura de um modelo salvo.

**Continue assim!**